In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import os

# --- 0. Carregamento do DataFrame Consolidado ---


In [2]:
# Define o caminho para a pasta 'processed'
processed_data_path = '../data/processed/'
file_path = os.path.join(processed_data_path, 'df_eda_consolidated.csv')

try:
    df = pd.read_csv(file_path)
    print("DataFrame carregado com sucesso!")
except FileNotFoundError:
    print(f"Erro: Arquivo '{file_path}' não encontrado.")
    print("Certifique-se de que o notebook '01_data_understanding.ipynb' foi executado e salvou 'df_eda_consolidated.csv'.")
    exit() # Interrompe a execução se o arquivo não for encontrado


DataFrame carregado com sucesso!


In [3]:
print("\n--- Primeiras 5 linhas do DataFrame ---")
df.head()


--- Primeiras 5 linhas do DataFrame ---


,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,target,sector,year_established,revenue,employees,office_location,subsidiary_of,series,sales_price,manager,regional_office
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Engaging,2016-10-20,2017-03-01,1054.0,0,retail,2001.0,718.62,2448.0,United States,NaN,GTX,1096.0,Dustin Brinkmann,Central
1,Z063OYW0,Darcel Schlecht,GTXPro,Isdom,Prospecting,2016-10-25,2017-03-11,4514.0,0,medical,2002.0,3178.24,4540.0,United States,NaN,NaN,NaN,Melvin Marxen,Central
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Engaging,2016-10-25,2017-03-07,50.0,0,retail,2001.0,718.62,2448.0,United States,NaN,MG,55.0,Melvin Marxen,Central
3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Engaging,2016-10-25,2017-03-09,588.0,0,software,1998.0,2714.90,2641.0,United States,Acme Corporation,GTX,550.0,Dustin Brinkmann,Central
4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Engaging,2016-10-25,2017-03-02,517.0,0,services,1982.0,792.46,1299.0,United States,NaN,GTX,550.0,Summer Sewald,West


In [4]:
print("\n--- Informações do DataFrame ---")
print(df.info())
print("\n--- Fim do carregamento ---")


--- Informações do DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   opportunity_id    8800 non-null   object 
 1   sales_agent       8800 non-null   object 
 2   product           8800 non-null   object 
 3   account           7375 non-null   object 
 4   deal_stage        8800 non-null   object 
 5   engage_date       8300 non-null   object 
 6   close_date        6711 non-null   object 
 7   close_value       7487 non-null   float64
 8   target            8800 non-null   int64  
 9   sector            7375 non-null   object 
 10  year_established  7375 non-null   float64
 11  revenue           7375 non-null   float64
 12  employees         7375 non-null   float64
 13  office_location   7375 non-null   object 
 14  subsidiary_of     1292 non-null   object 
 15  series            7320 non-null   object 
 16  sales_pr

# --- 1. Tratamento de Valores Ausentes ---

In [5]:
print("\n--- Contagem e Porcentagem de Valores Ausentes Antes do Preenchimento ---")
missing_data = df.isnull().sum()
missing_percentage = (missing_data / len(df) * 100).sort_values(ascending=False)
print(pd.DataFrame({'Missing Count': missing_data, 'Missing %': missing_percentage}))


--- Contagem e Porcentagem de Valores Ausentes Antes do Preenchimento ---
                  Missing Count  Missing %
account                    1425  16.193182
close_date                 2089  23.738636
close_value                1313  14.920455
deal_stage                    0   0.000000
employees                  1425  16.193182
engage_date                 500   5.681818
manager                       0   0.000000
office_location            1425  16.193182
opportunity_id                0   0.000000
product                       0   0.000000
regional_office               0   0.000000
revenue                    1425  16.193182
sales_agent                   0   0.000000
sales_price                1480  16.818182
sector                     1425  16.193182
series                     1480  16.818182
subsidiary_of              7508  85.318182
target                        0   0.000000
year_established           1425  16.193182


## 1.1. Preenchimento de Valores Ausentes Categóricos

In [6]:
# Preenche NaNs em colunas categóricas com um valor 'Unknown' ou 'Not_Subsidiary'
df['subsidiary_of'] = df['subsidiary_of'].fillna('Not_Subsidiary')
df['sector'] = df['sector'].fillna('Unknown_Sector')
df['office_location'] = df['office_location'].fillna('Unknown_Location')
# 'account' é um ID, se fosse para ser tratado como categoria, 'Unknown_Account' seria uma opção.
# No entanto, como será removido depois, o preenchimento aqui é apenas para consistência, se necessário.
df['account'] = df['account'].fillna('Unknown_Account')
df['series'] = df['series'].fillna('Unknown_Series')

## 1.2. Preenchimento de Valores Ausentes Numéricos

In [7]:
# Preenche NaNs em colunas numéricas com a mediana.
# 'close_value' já foi tratado no 'df_eda_consolidated.csv' e não deve ter NaNs significativos.
median_revenue = df['revenue'].median()
median_employees = df['employees'].median()
median_year_established = df['year_established'].median()
median_sales_price = df['sales_price'].median()
median_close_value = df['close_value'].median()

df['revenue'] = df['revenue'].fillna(median_revenue)
df['employees'] = df['employees'].fillna(median_employees)
df['year_established'] = df['year_established'].fillna(median_year_established)
df['sales_price'] = df['sales_price'].fillna(median_sales_price)
df['close_value'] = df['close_value'].fillna(median_close_value)

In [8]:
print("\n--- Porcentagem de Valores Ausentes por Coluna APÓS PREENCHIMENTO ---")
missing_data_after_fill = df.isnull().sum()
missing_percentage_after_fill = (missing_data_after_fill / len(df) * 100).sort_values(ascending=False)
print(pd.DataFrame({'Missing Count': missing_data_after_fill, 'Missing %': missing_percentage_after_fill}))


--- Porcentagem de Valores Ausentes por Coluna APÓS PREENCHIMENTO ---
                  Missing Count  Missing %
account                       0   0.000000
close_date                 2089  23.738636
close_value                   0   0.000000
deal_stage                    0   0.000000
employees                     0   0.000000
engage_date                 500   5.681818
manager                       0   0.000000
office_location               0   0.000000
opportunity_id                0   0.000000
product                       0   0.000000
regional_office               0   0.000000
revenue                       0   0.000000
sales_agent                   0   0.000000
sales_price                   0   0.000000
sector                        0   0.000000
series                        0   0.000000
subsidiary_of                 0   0.000000
target                        0   0.000000
year_established              0   0.000000


# --- 2. Engenharia de Features de Data/Duração ---

## 2.1. Conversão de Datas para datetime

In [9]:
# Garante que as colunas de data estejam no formato correto para cálculos.
df['engage_date'] = pd.to_datetime(df['engage_date'], errors='coerce')
df['close_date'] = pd.to_datetime(df['close_date'], errors='coerce')

## 2.2. Criação da feature 'opportunity_duration_days'

In [10]:
# Calcula a duração em dias. Valores NaT (Not a Time) resultarão em NaN na duração.
df['opportunity_duration_days'] = (df['close_date'] - df['engage_date']).dt.days

# Tratamento de NaNs e valores não positivos em 'opportunity_duration_days'
# Usaremos a mediana das durações **válidas e positivas** para imputação.
# Isso garante que a feature seja significativa para o modelo.
valid_durations = df['opportunity_duration_days'][df['opportunity_duration_days'] > 0]
if not valid_durations.empty:
    median_positive_duration = valid_durations.median()
    # Preenche NaNs e valores <= 0 com a mediana das durações válidas e positivas
    df['opportunity_duration_days'] = df['opportunity_duration_days'].apply(
        lambda x: median_positive_duration if pd.isna(x) or x <= 0 else x
    )
else:
    # Caso extremo: se não houver durações válidas e positivas, preenche com 0 ou outro valor que indique ausência.
    # Neste cenário, 0 é um fallback seguro, indicando uma duração mínima ou desconhecida.
    df['opportunity_duration_days'] = df['opportunity_duration_days'].fillna(0).apply(lambda x: 0 if x <= 0 else x)


In [11]:
print("\n--- Informações do DataFrame após engenharia de duração ---")
print(df[['engage_date', 'close_date', 'opportunity_duration_days']].info())


--- Informações do DataFrame após engenharia de duração ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   engage_date                8300 non-null   datetime64[ns]
 1   close_date                 6711 non-null   datetime64[ns]
 2   opportunity_duration_days  8800 non-null   float64       
dtypes: datetime64[ns](2), float64(1)
memory usage: 206.4 KB
None


In [12]:
print("\n--- Estatísticas descritivas da Duração da Oportunidade (em dias) ---")
print(df['opportunity_duration_days'].describe())


--- Estatísticas descritivas da Duração da Oportunidade (em dias) ---
count    8800.000000
mean       47.276705
std        35.876598
min         1.000000
25%        10.000000
50%        45.000000
75%        76.000000
max       138.000000
Name: opportunity_duration_days, dtype: float64


In [13]:
print("\n--- Porcentagem de Valores Ausentes após criação e tratamento de 'opportunity_duration_days' ---")
missing_after_duration = df.isnull().sum()
missing_percentage_after_duration = (missing_after_duration / len(df) * 100).sort_values(ascending=False)
print(pd.DataFrame({'Missing Count': missing_after_duration, 'Missing %': missing_percentage_after_duration}))


--- Porcentagem de Valores Ausentes após criação e tratamento de 'opportunity_duration_days' ---
                           Missing Count  Missing %
account                                0   0.000000
close_date                          2089  23.738636
close_value                            0   0.000000
deal_stage                             0   0.000000
employees                              0   0.000000
engage_date                          500   5.681818
manager                                0   0.000000
office_location                        0   0.000000
opportunity_duration_days              0   0.000000
opportunity_id                         0   0.000000
product                                0   0.000000
regional_office                        0   0.000000
revenue                                0   0.000000
sales_agent                            0   0.000000
sales_price                            0   0.000000
sector                                 0   0.000000
series            

In [14]:
print("Colunas do DataFrame final antes do OHE e escalonamento:")
print(df.columns.tolist())

Colunas do DataFrame final antes do OHE e escalonamento:
['opportunity_id', 'sales_agent', 'product', 'account', 'deal_stage', 'engage_date', 'close_date', 'close_value', 'target', 'sector', 'year_established', 'revenue', 'employees', 'office_location', 'subsidiary_of', 'series', 'sales_price', 'manager', 'regional_office', 'opportunity_duration_days']


In [15]:
# Para verificar rapidamente os tipos e NaNs (deve estar tudo limpo já):
print("\nInfo do DataFrame final antes do OHE e escalonamento:")
df.info()


Info do DataFrame final antes do OHE e escalonamento:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   opportunity_id             8800 non-null   object        
 1   sales_agent                8800 non-null   object        
 2   product                    8800 non-null   object        
 3   account                    8800 non-null   object        
 4   deal_stage                 8800 non-null   object        
 5   engage_date                8300 non-null   datetime64[ns]
 6   close_date                 6711 non-null   datetime64[ns]
 7   close_value                8800 non-null   float64       
 8   target                     8800 non-null   int64         
 9   sector                     8800 non-null   object        
 10  year_established           8800 non-null   float64       
 11  revenue       

# --- 3. Preparação para o Pipeline: Definição de Features ---

In [16]:
columns_to_drop_for_X = ['opportunity_id', 'account', 'engage_date', 'close_date', 'deal_stage']
existing_cols_to_drop = [col for col in columns_to_drop_for_X if col in df.columns]
df_final = df.drop(columns=existing_cols_to_drop, errors='ignore')

print(f"\n--- Colunas removidas para formação de X: {existing_cols_to_drop} ---")
print(f"Número de colunas após remoção: {df_final.shape[1]}")




--- Colunas removidas para formação de X: ['opportunity_id', 'account', 'engage_date', 'close_date', 'deal_stage'] ---
Número de colunas após remoção: 15


In [17]:
# Salvar df_final
# Cria o diretório 'processed' se não existir
processed_data_output_path = '../data/processed/'
if not os.path.exists(processed_data_output_path):
    os.makedirs(processed_data_output_path)
    print(f"Diretório '{processed_data_output_path}' criado.")

df_final_filename = os.path.join(processed_data_output_path, 'df_processed_for_modeling.csv')
df_final.to_csv(df_final_filename, index=False)
print(f"\n--- DataFrame final processado salvo em: '{df_final_filename}' ---")



--- DataFrame final processado salvo em: '../data/processed/df_processed_for_modeling.csv' ---


# --- 4. Construção do Pipeline de Pré-processamento ---

In [18]:
X_preprocessor_input = df_final.drop('target', axis=1)
y_target = df_final['target']

print(f"Dimensões de X (input para o pre-processador): {X_preprocessor_input.shape}")
print(f"Dimensões de y: {y_target.shape}")

Dimensões de X (input para o pre-processador): (8800, 14)
Dimensões de y: (8800,)


## 4.1. Definição das colunas para cada tipo de transformação

In [19]:
numeric_features = [
    'close_value',
    'year_established',
    'revenue',
    'employees',
    'sales_price',
    'opportunity_duration_days'
]

In [20]:
categorical_features = [
    'sales_agent',
    'product',
    'sector',
    'office_location',
    'subsidiary_of',
    'series',
    'manager',
    'regional_office'
]

In [21]:
# Verificação de que as features existem em X_preprocessor_input
missing_numeric = [f for f in numeric_features if f not in X_preprocessor_input.columns]
missing_categorical = [f for f in categorical_features if f not in X_preprocessor_input.columns]
if missing_numeric or missing_categorical:
    raise ValueError(f"Features missing from X_preprocessor_input: Numeric: {missing_numeric}, Categorical: {missing_categorical}")

print(f"\n--- Features numéricas para escalonamento: {numeric_features} ---")
print(f"--- Features categóricas para One-Hot Encoding: {categorical_features} ---")



--- Features numéricas para escalonamento: ['close_value', 'year_established', 'revenue', 'employees', 'sales_price', 'opportunity_duration_days'] ---
--- Features categóricas para One-Hot Encoding: ['sales_agent', 'product', 'sector', 'office_location', 'subsidiary_of', 'series', 'manager', 'regional_office'] ---


## 4.2. Criação do Pré-processador (ColumnTransformer)

In [22]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_features)
    ],
    remainder='passthrough'
)

## 4.3. Treinar o pre-processador (ColumnTransformer)

In [23]:
# Ele precisa ser ajustado nos dados de treino para aprender as escalas e categorias.
# Para isso, usaremos o X_preprocessor_input completo para ajustá-lo.
print("\n--- Ajustando o Pipeline de Pré-processamento (ColumnTransformer) ---")
preprocessor.fit(X_preprocessor_input) # Ajusta o pre-processador em TODOS os dados de X

print("--- Pipeline de Pré-processamento ajustado com sucesso! ---")


--- Ajustando o Pipeline de Pré-processamento (ColumnTransformer) ---
--- Pipeline de Pré-processamento ajustado com sucesso! ---


# --- 5. Salvamento do Pipeline de Pré-processamento ---

In [24]:
models_dir = '../models/'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"Diretório '{models_dir}' criado com sucesso.")

# Salvar APENAS o pipeline de pré-processamento
preprocessor_pipeline_filename = os.path.join(models_dir, 'preprocessor_pipeline.joblib')
joblib.dump(preprocessor, preprocessor_pipeline_filename) # Salva o ColumnTransformer diretamente

print(f"\n--- Pipeline de Pré-processamento salvo com sucesso em: '{preprocessor_pipeline_filename}' ---")
print("--- Fim do Processamento de Dados para Pré-processador ---")


--- Pipeline de Pré-processamento salvo com sucesso em: '../models/preprocessor_pipeline.joblib' ---
--- Fim do Processamento de Dados para Pré-processador ---
